# Project: Investigate a Dataset - [Database_soccer]

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

### Dataset Description 

> In this project 1, I willl be analysing ultimate soccer dataset, which is an open-source dataset in kaggle. The dataset is a one .sql file comprising seven tables, each with different(unique) but interrelated features. First, Country table has 11 European countries. Second, league table has 11 lead championship names. The country and league tables are related by their ID. Third, match table has over 25, 000 matches for different seasons as well as betting odds from upto 10 providers. The match table is also related to the previous tables by country_id. in the rows and 2 columns id and name
I have check the shape of table to determine the nummber of rows and columns.


### Question(s) for Analysis
1. What teams improved the most over the time period? 
2. Which players had the most penalties? 
3. Which was the the most preferred leg for penalty-takers in 2016 among the players who scored more than the mean penalties in that year?

In [1144]:
# import statements for all of the packages to be used.

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


%matplotlib inline

I want to create a function that would read csv and load for very dataset to a name variable

In [1145]:
# creating function to load data
path='C:/Users/Davie/Desktop/Data/'
def load_data(name, table_name):
    name=pd.read_csv(path + 'Database_Soccer/'+ table_name) # reads the csv file and stores in the dataframe name
    return name

displaying few lines of each dataset from the soccer database

In [1146]:
country=load_data('country', 'Country.csv')# country data table
country.head()

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [1147]:
league=load_data('league', 'League.csv')# league data table
league.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


From the league name above, the league name for Germany is confusing. Germany 1. could mean there are a number of German countries, so it should be change to Germany Bundesliga 1

In [1148]:
match=load_data('match', 'Match.csv')# match data table
match.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


missing data in match table

In [1149]:
player=load_data('player', 'Player.csv')# player data table
player.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


The birthday contains time at 00:00:00, which could be removed to contain only year, month and date

In [1150]:
player_attr=load_data('player_attr', 'Player_Attributes.csv') # player attributes data table
player_attr.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


The time 00:00:00 can be removed

In [1151]:
team=load_data('team', 'Team.csv')# team data table
team.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


The column team_fifa_api_id could insignificant because there is already team_api_id

In [1152]:
team_attr=load_data('team_attr', 'Team_Attributes.csv')# team attributes data table
team_attr.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


I want again to develop a function that I will be using in inspecting the datasets for missing data, getting descriptive statistics, dimensions, and features' data types

In [1153]:
# creating function for inspecting the datasets
def wrangles (tbl_name):
    
    inf=tbl_name.info(); # inspecting data types and instances with missing data 
    dim=tbl_name.shape; # inspecting dimensions of the dataset
    desc=tbl_name.describe(); # getting descriptive statistics
        
    return inf, dim, desc

In [1154]:
# country dataset
wrangles(country)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      11 non-null     int64 
 1   name    11 non-null     object
dtypes: int64(1), object(1)
memory usage: 304.0+ bytes


(None,
 (11, 2),
                  id
 count     11.000000
 mean   12452.090909
 std     8215.308472
 min        1.000000
 25%     6289.000000
 50%    13274.000000
 75%    18668.000000
 max    24558.000000)

There is no need of getting the mean for ids, since the names are string we can just unique values and counts 

In [1155]:
country.name.describe()

count          11
unique         11
top       Belgium
freq            1
Name: name, dtype: object

In [1156]:
# league data
wrangles(league)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          11 non-null     int64 
 1   country_id  11 non-null     int64 
 2   name        11 non-null     object
dtypes: int64(2), object(1)
memory usage: 392.0+ bytes


(None,
 (11, 3),
                  id    country_id
 count     11.000000     11.000000
 mean   12452.090909  12452.090909
 std     8215.308472   8215.308472
 min        1.000000      1.000000
 25%     6289.000000   6289.000000
 50%    13274.000000  13274.000000
 75%    18668.000000  18668.000000
 max    24558.000000  24558.000000)

There is no need of getting the mean for ids, since the names are string we can just unique values and counts

In [1157]:
league.name.describe()

count                         11
unique                        11
top       Belgium Jupiler League
freq                           1
Name: name, dtype: object

In [1158]:
# match data
wrangles(match)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25979 entries, 0 to 25978
Columns: 115 entries, id to BSA
dtypes: float64(96), int64(9), object(10)
memory usage: 22.8+ MB


(None,
 (25979, 115),
                  id    country_id     league_id         stage  match_api_id  \
 count  25979.000000  25979.000000  25979.000000  25979.000000  2.597900e+04   
 mean   12990.000000  11738.630317  11738.630317     18.242773  1.195429e+06   
 std     7499.635658   7553.936759   7553.936759     10.407354  4.946279e+05   
 min        1.000000      1.000000      1.000000      1.000000  4.831290e+05   
 25%     6495.500000   4769.000000   4769.000000      9.000000  7.684365e+05   
 50%    12990.000000  10257.000000  10257.000000     18.000000  1.147511e+06   
 75%    19484.500000  17642.000000  17642.000000     27.000000  1.709852e+06   
 max    25979.000000  24558.000000  24558.000000     38.000000  2.216672e+06   
 
        home_team_api_id  away_team_api_id  home_team_goal  away_team_goal  \
 count      25979.000000      25979.000000    25979.000000    25979.000000   
 mean        9984.371993       9984.475115        1.544594        1.160938   
 std        14087.4537

missing data for column home_player_x1

In [1159]:
match.dtypes

id              int64
country_id      int64
league_id       int64
season         object
stage           int64
               ...   
GBD           float64
GBA           float64
BSH           float64
BSD           float64
BSA           float64
Length: 115, dtype: object

the date has been stored as a string

In [1160]:
# player data
wrangles(player)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11060 entries, 0 to 11059
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  11060 non-null  int64  
 1   player_api_id       11060 non-null  int64  
 2   player_name         11060 non-null  object 
 3   player_fifa_api_id  11060 non-null  int64  
 4   birthday            11060 non-null  object 
 5   height              11060 non-null  float64
 6   weight              11060 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 605.0+ KB


(None,
 (11060, 7),
                  id  player_api_id  player_fifa_api_id        height  \
 count  11060.000000   11060.000000        11060.000000  11060.000000   
 mean    5537.511392  156582.427215       165664.910488    181.867445   
 std     3197.692647  160713.700624        58649.928360      6.369201   
 min        1.000000    2625.000000            2.000000    157.480000   
 25%     2767.750000   35555.500000       151889.500000    177.800000   
 50%     5536.500000   96619.500000       184671.000000    182.880000   
 75%     8306.250000  212470.500000       203883.250000    185.420000   
 max    11075.000000  750584.000000       234141.000000    208.280000   
 
              weight  
 count  11060.000000  
 mean     168.380289  
 std       14.990217  
 min      117.000000  
 25%      159.000000  
 50%      168.000000  
 75%      179.000000  
 max      243.000000  )

birthday column stored as string

In [1161]:
# player attributes data
wrangles(player_attr)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

(None,
 (183978, 42),
                  id  player_fifa_api_id  player_api_id  overall_rating  \
 count  183978.00000       183978.000000  183978.000000   183142.000000   
 mean    91989.50000       165671.524291  135900.617324       68.600015   
 std     53110.01825        53851.094769  136927.840510        7.041139   
 min         1.00000            2.000000    2625.000000       33.000000   
 25%     45995.25000       155798.000000   34763.000000       64.000000   
 50%     91989.50000       183488.000000   77741.000000       69.000000   
 75%    137983.75000       199848.000000  191080.000000       73.000000   
 max    183978.00000       234141.000000  750584.000000       94.000000   
 
            potential       crossing      finishing  heading_accuracy  \
 count  183142.000000  183142.000000  183142.000000     183142.000000   
 mean       73.460353      55.086883      49.921078         57.266023   
 std         6.592271      17.242135      19.038705         16.488905   
 min     

dates stored as string

In [1162]:
# team data
wrangles(team)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                299 non-null    int64  
 1   team_api_id       299 non-null    int64  
 2   team_fifa_api_id  288 non-null    float64
 3   team_long_name    299 non-null    object 
 4   team_short_name   299 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 11.8+ KB


(None,
 (299, 5),
                  id    team_api_id  team_fifa_api_id
 count    299.000000     299.000000        288.000000
 mean   23735.301003   12340.521739      21534.305556
 std    15167.914719   25940.411135      42456.439408
 min        1.000000    1601.000000          1.000000
 25%     9552.500000    8349.000000        178.750000
 50%    22805.000000    8655.000000        673.500000
 75%    36250.500000    9886.500000       1910.750000
 max    51606.000000  274581.000000     112513.000000)

Missing data for team_fifa_api_id, though this might be insignificant since there is already team_api_id

In [1163]:
# team attributes data
wrangles(team_attr)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              1458 non-null   int64  
 1   team_fifa_api_id                1458 non-null   int64  
 2   team_api_id                     1458 non-null   int64  
 3   date                            1458 non-null   object 
 4   buildUpPlaySpeed                1458 non-null   int64  
 5   buildUpPlaySpeedClass           1458 non-null   object 
 6   buildUpPlayDribbling            489 non-null    float64
 7   buildUpPlayDribblingClass       1458 non-null   object 
 8   buildUpPlayPassing              1458 non-null   int64  
 9   buildUpPlayPassingClass         1458 non-null   object 
 10  buildUpPlayPositioningClass     1458 non-null   object 
 11  chanceCreationPassing           1458 non-null   int64  
 12  chanceCreationPassingClass      14

(None,
 (1458, 25),
                 id  team_fifa_api_id    team_api_id  buildUpPlaySpeed  \
 count  1458.000000       1458.000000    1458.000000       1458.000000   
 mean    729.500000      17706.982167    9995.727023         52.462277   
 std     421.032659      39179.857739   13264.869900         11.545869   
 min       1.000000          1.000000    1601.000000         20.000000   
 25%     365.250000        110.000000    8457.750000         45.000000   
 50%     729.500000        485.000000    8674.000000         52.000000   
 75%    1093.750000       1900.000000    9904.000000         62.000000   
 max    1458.000000     112513.000000  274581.000000         80.000000   
 
        buildUpPlayDribbling  buildUpPlayPassing  chanceCreationPassing  \
 count            489.000000         1458.000000            1458.000000   
 mean              48.607362           48.490398              52.165295   
 std                9.678290           10.896101              10.360793   
 min        

Missing data for buildUpPlayDribbling


### Data Cleaning

First, I want to create functions that will help me drop duplicates, merge two datasets, change data type, remove missing rows, drop unnecessary columns, then proceed to to merge the country data to that for league. 
I will correct the league name for Germany 1. Bundesliga to Germany Bundesliga 1.
I will also change the name column for both the country data and league data, and also make the datafrmaes have the same dimensions and finally merge the two dataframes into country_league data using the country id as the key

In [1164]:
# creating function to rename columns in a data frame
def col_rename(col_renamed_data, col_old, col_new):
    if len(col_old)==2: # checks if there are two columns to be renamed
        col_renamed_data.rename(columns={col_old[0]:col_new[0], col_old[1]:col_new[1]}, inplace=True) # renames the two columns in the dataset
    else:
        col_renamed_data.rename(columns={col_old:col_new}, inplace=True) # renames if there is only one column to be renamed
        
    return col_renamed_data

In [1165]:
# creating a function drop columns
def drop_cols(drop_col_data, col_name):
    drop_col_data.drop(col_name, axis=1, inplace=True) # removing columns
        
    return drop_col_data

In [1166]:
# creating a function to remove row missing values
def remove_row_missing_values(na_data):
    na_data.dropna(axis=0, how='any', inplace=True) # removing all rows with missing values
        
    return na_data

In [1167]:
# creating a function to remove duplicate rows
def remove_duplicates(dup_data, col):
    if col=='':
        dup_data.drop_duplicates(inplace=True) # remove all duplicate rows
        
    else:
        dup_data.dropna(subset=[col], inplace=True) # removing rows based on column duplicate values
        
    return dup_data

In [1168]:
# creating a function to change date column from string to datetime
def changed_type(changed_type_data, col_type):
    changed_type_data[col_type]=changed_type_data[col_type].astype('str')  # converting to string
    changed_type_data[col_type]=changed_type_data[col_type].str.extract(r'(\d{4}-\d{2}-\d{2})') # extracting the date
    changed_type_data[col_type] = pd.to_datetime(changed_type_data[col_type], format='%Y-%m-%d') # converting to datetime

    return changed_type_data

In [1169]:
# creating a function to filter some columns
def filter_col(f_data, col):
    df=f_data.filter(col) # filters the columns
        
    return df

In [1170]:
# creating a function to to merge two data frames using inner because i dont want do keep duplicates
def merging_data(data1, data2, on_col):
    df=data1.merge(data2, on =on_col, how='inner')
        
    return df

In [1171]:
# correcting the league name for Germany 1. Bundesliga
league.replace(to_replace='Germany 1. Bundesliga', value='Germany Bundesliga 1', inplace=True)

In [1172]:
# changing both id and name columns for the country data
country=col_rename(country, col_old=['name', 'id'], col_new=['country_name', 'country_id']) 
country 

,country_id,country_name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [1173]:
# changing the name column and id column  for the league data
league=col_rename(league, col_old=['name', 'id'], col_new=['league_name', 'league_id']) 
league

,league_id,country_id,league_name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany Bundesliga 1
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [1174]:
# merging the two dataframes country and league
country_league_info=merging_data(country, league, on_col ='country_id')
country_league_info

,country_id,country_name,league_id,league_name
0,1,Belgium,1,Belgium Jupiler League
1,1729,England,1729,England Premier League
2,4769,France,4769,France Ligue 1
3,7809,Germany,7809,Germany Bundesliga 1
4,10257,Italy,10257,Italy Serie A
5,13274,Netherlands,13274,Netherlands Eredivisie
6,15722,Poland,15722,Poland Ekstraklasa
7,17642,Portugal,17642,Portugal Liga ZON Sagres
8,19694,Scotland,19694,Scotland Premier League
9,21518,Spain,21518,Spain LIGA BBVA


Second, I want to merge the player data to player attributes data into player_info dataframe.
I will use either the player_api_id or player_fifa_api_id as the keys and drop the id columns in both datasets


In [1175]:
# checking the number of unique values of player_api_id on payer and player attribute data
player.player_api_id.nunique()==player_attr.player_api_id.nunique()

True

In [1176]:
# checking the number of unique values of  player_fifa_api_id on player and player attribute data
player.player_fifa_api_id.nunique()==player_attr.player_fifa_api_id.nunique()

False

Since the number of unique values of the player_fifa_api_id are not the same in both dataframes, I will just use both of them as the key because both the ids might be important in merging this data with another one

In [1177]:
# dropping id column from player dataframe
drop_cols(player, col_name='id')
player.drop_duplicates(subset='player_api_id')

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...
11055,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [1178]:
# dropping id column from player_attr dataframe
drop_cols(player_attr, col_name='id')
player_attr.shape

(183978, 41)

In [1179]:
# removing duplicates for player attribute data
remove_duplicates(player_attr, col='')
player_attr.sort_values(['player_api_id', 'date']) # sorting the players by id and date

,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
139856,148544,2625,2007-02-22 00:00:00,63.0,64.0,right,high,medium,48.0,48.0,...,56.0,46.0,64.0,66.0,63.0,14.0,11.0,67.0,9.0,10.0
139855,148544,2625,2007-08-30 00:00:00,63.0,64.0,right,high,medium,48.0,48.0,...,56.0,46.0,64.0,66.0,63.0,14.0,24.0,67.0,24.0,24.0
139854,148544,2625,2008-08-30 00:00:00,60.0,64.0,right,high,medium,48.0,48.0,...,56.0,46.0,64.0,66.0,63.0,14.0,24.0,67.0,24.0,24.0
139853,148544,2625,2010-08-30 00:00:00,60.0,64.0,right,high,medium,48.0,48.0,...,56.0,69.0,64.0,66.0,63.0,12.0,11.0,6.0,8.0,8.0
139852,148544,2625,2011-02-22 00:00:00,59.0,63.0,right,high,medium,52.0,47.0,...,55.0,66.0,63.0,63.0,62.0,12.0,11.0,6.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147410,225462,750435,2016-02-04 00:00:00,56.0,70.0,right,medium,low,35.0,57.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
147409,225462,750435,2016-02-11 00:00:00,56.0,70.0,right,medium,low,35.0,57.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
147408,225462,750435,2016-04-14 00:00:00,60.0,74.0,right,medium,low,35.0,57.0,...,58.0,61.0,18.0,19.0,21.0,9.0,10.0,8.0,10.0,11.0
8086,234141,750584,2007-02-22 00:00:00,58.0,66.0,right,medium,medium,47.0,39.0,...,47.0,52.0,56.0,66.0,62.0,8.0,9.0,13.0,11.0,13.0


In [1180]:
# joining the two dataframes on player_api_id and player_fifa_api_id using inner join because i dont want the unmatched rows
player_info=merging_data(player, player_attr, on_col =['player_api_id', 'player_fifa_api_id'])
player_info

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2016-02-18 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-11-19 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-09-21 00:00:00,62.0,66.0,right,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-03-20 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2007-02-22 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183761,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2009-08-30 00:00:00,83.0,85.0,right,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183762,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2009-02-22 00:00:00,78.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183763,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2008-08-30 00:00:00,77.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183764,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2007-08-30 00:00:00,78.0,81.0,right,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


I now want to examine the player_info dataframe, check for missing values, dimensions of each column, data types of each column as well as duplcate values

In [1181]:
player_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183766 entries, 0 to 183765
Data columns (total 45 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   player_api_id        183766 non-null  int64  
 1   player_name          183766 non-null  object 
 2   player_fifa_api_id   183766 non-null  int64  
 3   birthday             183766 non-null  object 
 4   height               183766 non-null  float64
 5   weight               183766 non-null  int64  
 6   date                 183766 non-null  object 
 7   overall_rating       183016 non-null  float64
 8   potential            183016 non-null  float64
 9   preferred_foot       183016 non-null  object 
 10  attacking_work_rate  180622 non-null  object 
 11  defensive_work_rate  183016 non-null  object 
 12  crossing             183016 non-null  float64
 13  finishing            183016 non-null  float64
 14  heading_accuracy     183016 non-null  float64
 15  short_passing    

From the above uotput, there are several missing values, the data type for birthday and date are all strings. I have also noted that attacking_work_rate has the least number of rows hence maximum number of missing values

In [1182]:
# getting the uniques for the coulmn attacking_work_rate
player_info.attacking_work_rate.unique()

array(['medium', 'high', nan, 'low', 'None', 'le', 'norm', 'stoc', 'y'],
      dtype=object)

In [1183]:
# getting the uniques for the coulmn attacking_work_rate
player_info.defensive_work_rate.unique()

array(['medium', 'high', 'low', '_0', nan, '5', 'ean', 'o', '1', 'ormal',
       '7', '2', '8', '4', 'tocky', '0', '3', '6', '9', 'es'],
      dtype=object)

From the output in above two cells, no information has been provided from the data description about the meaning of None, norm, y, stoc, le, ornal, es, tocky, ean o, and the numbers 0, 1, 2, 3, 4, 5, 6, 7, 8, and 9 . Howerver, from inspection, I realize that there is some association in that all the numbers 0-9 and o on the defensive_work_rate relates to None  on the attacking_work_rate. Also, the following  pairs also relate: norm-ornal, y-es, stoc-tocky, le-ean. These could be change or transformed if additional information is provided or simply be dropped from the analysis

In [1184]:
attacking_none=player_info.query('attacking_work_rate=="None"')
attacking_none

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
675,194479,"Abdoulaye Diallo Sadio,22",204171,1990-12-28 00:00:00,182.88,168,2016-03-24 00:00:00,73.0,75.0,right,...,71.0,68.0,44.0,36.0,24.0,8.0,14.0,14.0,8.0,13.0
676,194479,"Abdoulaye Diallo Sadio,22",204171,1990-12-28 00:00:00,182.88,168,2016-02-25 00:00:00,72.0,74.0,right,...,71.0,68.0,44.0,36.0,24.0,8.0,14.0,14.0,8.0,13.0
677,194479,"Abdoulaye Diallo Sadio,22",204171,1990-12-28 00:00:00,182.88,168,2016-01-07 00:00:00,72.0,74.0,right,...,74.0,68.0,44.0,36.0,24.0,8.0,14.0,14.0,8.0,13.0
678,194479,"Abdoulaye Diallo Sadio,22",204171,1990-12-28 00:00:00,182.88,168,2015-09-21 00:00:00,72.0,74.0,right,...,74.0,68.0,44.0,36.0,24.0,8.0,14.0,14.0,8.0,13.0
679,194479,"Abdoulaye Diallo Sadio,22",204171,1990-12-28 00:00:00,182.88,168,2015-07-03 00:00:00,71.0,73.0,right,...,73.0,67.0,43.0,35.0,23.0,7.0,13.0,13.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183520,39494,Zlatko Dedic,141111,1984-10-05 00:00:00,182.88,170,2009-08-30 00:00:00,68.0,78.0,right,...,66.0,65.0,28.0,31.0,29.0,8.0,23.0,28.0,23.0,23.0
183521,39494,Zlatko Dedic,141111,1984-10-05 00:00:00,182.88,170,2009-02-22 00:00:00,66.0,78.0,right,...,66.0,65.0,28.0,31.0,29.0,8.0,23.0,28.0,23.0,23.0
183522,39494,Zlatko Dedic,141111,1984-10-05 00:00:00,182.88,170,2008-08-30 00:00:00,63.0,76.0,right,...,66.0,62.0,28.0,31.0,29.0,8.0,23.0,28.0,23.0,23.0
183523,39494,Zlatko Dedic,141111,1984-10-05 00:00:00,182.88,170,2007-08-30 00:00:00,64.0,78.0,right,...,66.0,62.0,28.0,31.0,29.0,8.0,23.0,25.0,23.0,23.0


In [1185]:
# comparing length of numbers 0-9 and o in the defensive_work_rate relates to None in the attacking_work_rate

attacking_none.attacking_work_rate.value_counts()==attacking_none.defensive_work_rate.value_counts().sum()

None    True
Name: attacking_work_rate, dtype: bool

- None: This likely means no specific attacking work rate is assigned. Treat it as a default or neutral value - medium.
- le: This could be an abbreviation for "less effort" or something similar. map it to a "Low" attacking work rate.
- norm: This probably stands for "normal.map it to a "Medium" attacking work rate.
- stoc: This might be short for "stock" or default settings. map it to "Medium."
- y: This is unclear, but it might be a placeholder or a specific setting in a custom game mode. Remove
- I want to drop all rows with nan and duplicate rows

In [1186]:
attacking_work_rate_mapping={
    'None': 'medium',
    'le': 'low',
    'norm': 'medium',
    'stoc': 'medium',
    'y': np.NAN
}
player_info['attacking_work_rate']=player_info['attacking_work_rate'].replace(attacking_work_rate_mapping)

In [1187]:
player_info['attacking_work_rate'].unique()

array(['medium', 'high', nan, 'low'], dtype=object)

- 5, 1, 7, 2, 8, 4, 0, 3, 6, 9: mapping based on the context or frequency of these values.
- ean: Possibly a typo for "mean" or "lean." map it to "Medium" or "Low."
- o: Could be a typo or placeholder. Remove
- ormal: Likely a typo for "normal," which you can map to "Medium."
- tocky: Possibly a typo for "stocky," which might imply a strong defensive presence. map it to "High."
- es: Could be a typo for "yes" or another term. Remove

In [1188]:
defensive_work_rate_mapping = {
    '0': 'low',
    '1': 'low',
    '2': 'low',
    '3': 'low',
    '4': 'medium',
    '5': 'medium',
    '6': 'medium',
    '7': 'high',
    '8': 'high',
    '9': 'high',
    'ean':'low',
    'ormal': 'medium',
    'tocky': 'high',
    'o':np.nan,
    'es':np.nan
}
player_info['defensive_work_rate']=player_info['defensive_work_rate'].replace(defensive_work_rate_mapping)


In [1189]:
player_info['defensive_work_rate'].unique()

array(['medium', 'high', 'low', '_0', nan], dtype=object)

In [1190]:
# dropping all rows with missing data
remove_row_missing_values(player_info) 

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2016-02-18 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-11-19 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-09-21 00:00:00,62.0,66.0,right,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-03-20 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2007-02-22 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183761,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2009-08-30 00:00:00,83.0,85.0,right,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183762,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2009-02-22 00:00:00,78.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183763,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2008-08-30 00:00:00,77.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183764,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2007-08-30 00:00:00,78.0,81.0,right,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [1191]:
# dropping duplicate rows
remove_duplicates(player_info, col='')

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2016-02-18 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-11-19 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-09-21 00:00:00,62.0,66.0,right,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2015-03-20 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,2007-02-22 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183761,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2009-08-30 00:00:00,83.0,85.0,right,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183762,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2009-02-22 00:00:00,78.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183763,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2008-08-30 00:00:00,77.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183764,39902,Zvjezdan Misimovic,102359,1982-06-05 00:00:00,180.34,176,2007-08-30 00:00:00,78.0,81.0,right,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [1192]:
player_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178806 entries, 0 to 183765
Data columns (total 45 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   player_api_id        178806 non-null  int64  
 1   player_name          178806 non-null  object 
 2   player_fifa_api_id   178806 non-null  int64  
 3   birthday             178806 non-null  object 
 4   height               178806 non-null  float64
 5   weight               178806 non-null  int64  
 6   date                 178806 non-null  object 
 7   overall_rating       178806 non-null  float64
 8   potential            178806 non-null  float64
 9   preferred_foot       178806 non-null  object 
 10  attacking_work_rate  178806 non-null  object 
 11  defensive_work_rate  178806 non-null  object 
 12  crossing             178806 non-null  float64
 13  finishing            178806 non-null  float64
 14  heading_accuracy     178806 non-null  float64
 15  short_passing    

In [1193]:
player_info.attacking_work_rate.value_counts()

medium    127384
high       42775
low         8647
Name: attacking_work_rate, dtype: int64

In [1194]:
player_info.defensive_work_rate.value_counts()

medium    131615
high       27547
low        19644
Name: defensive_work_rate, dtype: int64

I want to change the data types for birthday and date from string to datetime

In [1195]:
# converting birthday column into datetime
changed_type(player_info, col_type='birthday')

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2016-02-18 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2015-11-19 00:00:00,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2015-09-21 00:00:00,62.0,66.0,right,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2015-03-20 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2007-02-22 00:00:00,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183761,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,2009-08-30 00:00:00,83.0,85.0,right,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183762,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,2009-02-22 00:00:00,78.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183763,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,2008-08-30 00:00:00,77.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183764,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,2007-08-30 00:00:00,78.0,81.0,right,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [1196]:
# converting date column into datetime
changed_type(player_info, col_type='date')

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2016-02-18,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2015-11-19,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2015-09-21,62.0,66.0,right,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2015-03-20,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2007-02-22,61.0,65.0,right,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183761,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,2009-08-30,83.0,85.0,right,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183762,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,2009-02-22,78.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183763,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,2008-08-30,77.0,80.0,right,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183764,39902,Zvjezdan Misimovic,102359,1982-06-05,180.34,176,2007-08-30,78.0,81.0,right,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [1197]:
player_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178806 entries, 0 to 183765
Data columns (total 45 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   player_api_id        178806 non-null  int64         
 1   player_name          178806 non-null  object        
 2   player_fifa_api_id   178806 non-null  int64         
 3   birthday             178806 non-null  datetime64[ns]
 4   height               178806 non-null  float64       
 5   weight               178806 non-null  int64         
 6   date                 178806 non-null  datetime64[ns]
 7   overall_rating       178806 non-null  float64       
 8   potential            178806 non-null  float64       
 9   preferred_foot       178806 non-null  object        
 10  attacking_work_rate  178806 non-null  object        
 11  defensive_work_rate  178806 non-null  object        
 12  crossing             178806 non-null  float64       
 13  finishing     

In [ ]:
player_info.drop_duplicates(subset='player_api_id', inplace=True)
player_info.head()

,player_api_id,player_name,player_fifa_api_id,birthday,height,weight,date,overall_rating,potential,preferred_foot,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,2016-02-18,67.0,71.0,right,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
5,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,2016-04-21,74.0,76.0,left,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
38,162549,Aaron Doran,186170,1991-05-13,170.18,163,2016-01-07,65.0,67.0,right,...,68.0,61.0,23.0,22.0,24.0,16.0,11.0,12.0,9.0,13.0
64,30572,Aaron Galindo,140161,1982-05-08,182.88,198,2016-04-21,69.0,69.0,right,...,54.0,37.0,72.0,71.0,68.0,15.0,12.0,13.0,12.0,11.0
87,23780,Aaron Hughes,17725,1979-11-08,182.88,154,2015-12-24,70.0,70.0,right,...,41.0,45.0,75.0,73.0,71.0,8.0,6.0,16.0,12.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183683,192132,Zouhaier Dhaouadhi,195840,1988-01-01,180.34,159,2016-01-21,64.0,64.0,left,...,35.0,76.0,39.0,26.0,24.0,16.0,10.0,12.0,8.0,8.0
183691,282473,Zouhair Feddal,205705,1989-01-01,190.50,172,2016-02-04,75.0,76.0,left,...,47.0,48.0,78.0,79.0,71.0,13.0,14.0,12.0,10.0,8.0
183712,26357,Zoumana Camara,2488,1979-04-03,182.88,168,2015-03-06,73.0,73.0,right,...,52.0,50.0,76.0,77.0,72.0,14.0,10.0,11.0,8.0,12.0
183725,111182,Zsolt Laczko,164680,1986-12-18,182.88,176,2014-09-18,63.0,67.0,left,...,49.0,53.0,57.0,61.0,61.0,9.0,8.0,5.0,5.0,8.0


Third, I want merge team data to team attributes data into into team info dataframe, check data types, missing values

In [ ]:
# checking the number of unique values of team_api_id in team and team attribute data
team.team_api_id.nunique()==team_attr.team_api_id.nunique()

In [ ]:
# checking the number of unique values of team_fifa_api_id in team and team attribute data
team.team_fifa_api_id.nunique()==team_attr.team_fifa_api_id.nunique()

From these outputs, it is clear that neither the team_api_id nor the team_fifa_api_id matches in the two datasets. I will therefore merge them on both the team_api_id and team_fifa_api_id as the keys

In [ ]:
team

In [ ]:
# dropping the id for team dataframe
drop_cols(team, col_name='id').drop_duplicates(subset='team_api_id')

In [ ]:
team.team_long_name.unique()

I want to correct the following team names: '1. FC Köln', '1. FC Nürnberg', '1. FSV Mainz 05','1. FC Kaiserslautern'

In [1009]:
# correcting some team names
team.team_long_name.replace(to_replace=['1. FC Köln', '1. FC Nürnberg', '1. FSV Mainz 05','1. FC Kaiserslautern'], 
                            value=['FC Köln', 'FC Nürnberg', 'FSV Mainz 05','FC Kaiserslautern'], inplace=True)

In [ ]:
team_attr.dropna(how='any').drop_duplicates(subset='team_api_id')

In [ ]:
# dropping the id for team attributes data
drop_cols(team_attr, col_name='id')

In [ ]:
# merging team to the team attribute
team_info=team.merge(team_attr, on =['team_api_id', 'team_fifa_api_id'], how='inner') #
team_info['team_fifa_api_id']=team_info['team_fifa_api_id'].astype(int)
team_info.head()

I will populate the NaN in the buildUpPlayDribbling column with the mean of the column. And finally drop duplcates in the final dataframe

In [1013]:
# getting the mean of buildUpPlayDribbling column
#mean=team_info.buildUpPlayDribbling.mean()

# filling the NaNs in the buildUpPlayDribbling by the mean
#team_info['buildUpPlayDribbling']=team_info['buildUpPlayDribbling'].fillna(mean)

In [ ]:
# converting to datetime
changed_type(team_info, col_type='date')

In [ ]:
# dropping the duplicate rows in the final team info merged data
team_info=team_info.copy()
drop_cols(team_info, col_name=['team_fifa_api_id',])
team_info.drop_duplicates(subset='team_api_id')
team_info.dropna(how='any', inplace=True)
#remove_duplicates(team_info, col='')
team_info.head(5)

Finally, I to examine and merge match data, country_league, and team data to form march info dataframe. I will check for duplicates, missing values and correct data types

In [ ]:
# checking the number of league_id unique values in the match and country_league_info dataframes
match.league_id.nunique()==country_league_info.league_id.nunique()

In [ ]:
# checking the number of country_id unique values in the match and country_league_info dataframes
match.country_id.nunique()==country_league_info.country_id.nunique()

I will merge match and country_league_info dataframes on league_id and country_id, so that I retain all the info about ids. I want to drop all the columns containing odds

In [ ]:
for i in match.columns:
    print(i)

I want to remove all the columns with CAPITAL LETTERS, odds columns

In [ ]:
# converting to datetime for the match data
changed_type(match, col_type='date').head(5)

In [ ]:
# dropping the duplicate rows from match data
remove_duplicates(match, col='').head(5)

In [ ]:
# dropping the columns containg odds and the match id
odd_cols = list(filter(lambda x: x.isupper(), match.columns))
drop_cols(match, col_name=odd_cols)
drop_cols(match, col_name='id')
match.head(5)

In [ ]:
match.columns

### checking the contents of some columns
```match.goal.value_counts() # goal```

From the above two cells, I realised that the coulmns goal, shoton, shotoff, foulcommit, card, cross, corner and possession contains information related to the web page but not realistic data.Checking through the nested infomation, I can't really make sense out of it since even the website link to the discription is not loading. Instead of deleting the columns with such issues, I will instead drop the columns containing unprocessed data

In [ ]:
match.columns
match_info=filter_col(match, col=['country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal'])

match_info.head(5)

In [ ]:
match_info['results'] = np.where(
    match_info['home_team_goal'] == match_info['away_team_goal'], 'draw',
    np.where(
        match_info['home_team_goal'] > match_info['away_team_goal'], 'home win', 'away win'
    )
)

match_info.head(5)

In [ ]:
# Merging the data
match_info=merging_data(country_league_info, match_info, on_col =['country_id', 'league_id'])
match_info

In [ ]:
match_info=match_info.drop(columns=['country_id', 'league_id'])
match_info.head(5)

In [ ]:
# melt home and away teams id_vars are columns not melted
# creating columns not melted
unmelted_columns= [x for x in match_info.columns if x not in ['home_team_api_id', 'away_team_api_id']]

# melting home and away teams api ids
melted_match=match_info.melt(id_vars=unmelted_columns, var_name='Location', value_name='Team')
melted_match.head(5)

In [1028]:
# cleaning up or changing the location values
melted_match['Location']=melted_match['Location'].replace({'home_team_api_id':'home', 'away_team_api_id':'away'})
melted_df=melted_match.copy()


### merging the team_info to the melted_match

In [ ]:
exclude_columns = ['team_long_name', 'team_short_name', 'date']
team_info = team_info.drop(columns=exclude_columns).drop_duplicates(subset='team_api_id')

team_info.head(5)


In [1030]:
team_info.rename(columns={'team_api_id':'Team'}, inplace=True)
team_s=team_info.copy()

In [ ]:

melted_match=pd.merge(melted_df, team_s, on ='Team', how='left')
#melted_match.drop_duplicates(subset='match_api_id' , inplace=True)
melted_match.head(5)


In [ ]:
# creating team dictionary with team api ids as the key and a longname as the value
team_dict=team.set_index('team_api_id')['team_long_name'].to_dict()
team_dict

In [ ]:
# cleaning up team i.e. using team dictionary to replace team api ids
melted_match['Team']=melted_match['Team'].map(team_dict)
melted_match.head(5)

In [ ]:
# creating goals column
melted_match['goals']=np.where(melted_match['Location']=='home',melted_match['home_team_goal'], melted_match['away_team_goal'])
melted_match.head(5)

In [ ]:
melted_match['pts'] = np.where(
    melted_match['results'] == 'draw', 1, 
    np.where(
        (melted_match['results'] == 'away win') & (melted_match['Location'] == 'home'), 0, 
        np.where(
            (melted_match['results'] == 'home win') & (melted_match['Location'] == 'away'), 0, 3
        )
    )
)

# Display the DataFrame
melted_match.head()


In [ ]:
melted_match.sort_values(by=['match_api_id', 'season'])
melted_match.drop_duplicates() #subset='match_api_id', inplace=True
melted_match.dropna(how='any')
melted_match.sort_values(by='match_api_id', ascending=True)

In [ ]:
# Goals For for the home team
melted_match['GF'] = np.where(
    # Case 1
    (melted_match['Location'] == 'home') & (melted_match['results'] == 'home win'),  
    melted_match['goals'],  
    np.where(
        # Case 2
        (melted_match['Location'] == 'home') & (melted_match['results'] == 'away win'),  
        melted_match['goals'],  

        np.where(
            # Case 3
            (melted_match['Location'] == 'away') & (melted_match['results'] == 'home win'),  
            melted_match['goals'],  

            np.where(
                # Case 4
                (melted_match['Location'] == 'home') & (melted_match['results'] == 'draw'),  
                melted_match['goals'],  

                np.where(
                    # Case 5
                    (melted_match['Location'] == 'away') & (melted_match['results'] == 'draw'),  
                    melted_match['goals'],  

                    np.where(
                        # Case 6
                        (melted_match['Location'] == 'away') & (melted_match['results'] == 'away win'),  
                        melted_match['goals'], 

                        0  # If none of the conditions are met, GF is 0 for the home team
                    )
                )
            )
        )
    )
)

melted_match.head(5)

In [ ]:
# Goals Against for the home team
melted_match['GA'] = np.where(
    # Case 1
    (melted_match['Location'] == 'home') & (melted_match['results'] == 'home win'),  
    melted_match['away_team_goal'], 

    np.where(
        # Case 2 
        (melted_match['Location'] == 'away') & (melted_match['results'] == 'home win'),  
        melted_match['home_team_goal'],  
        np.where(
            # Case 3
            (melted_match['Location'] == 'home') & (melted_match['results'] == 'draw'),  
            melted_match['away_team_goal'],  
            np.where(
                # Case 4
                (melted_match['Location'] == 'away') & (melted_match['results'] == 'draw'),  
                melted_match['home_team_goal'], 

                np.where(
                    # Case 5
                    (melted_match['Location'] == 'away') & (melted_match['results'] == 'away win'),  
                    melted_match['home_team_goal'],  

                    np.where(
                        # Case 6
                        (melted_match['Location'] == 'away') & (melted_match['results'] == 'home win'),  
                        melted_match['home_team_goal'],  

                        np.where(
                        # Case 7
                        (melted_match['Location'] == 'home') & (melted_match['results'] == 'away win'),  
                        melted_match['away_team_goal'],  

                        0  # Default case (if none of the above conditions are met
                        )

                    )
                )
            )
        )
    )
)

melted_match.head(5)

In [ ]:
# Calculate Goal Difference
melted_match['GD'] = melted_match['GF'] - melted_match['GA']
melted_match.head(5)

<a id='eda'></a>
## Exploratory Data Analysis

### creating a function to plot different visualizations

In [1040]:
def plot_visual(data, data2, visual_type):
    if visual_type=='barh':
        data.plot(kind='barh', rot=0, width=0.7, alpha=0.8, color='grey', figsize=[8,20] ) # i want to creat horizontal bars
            
    elif visual_type=='hist':
        fig,vis=plt.subplots(figsize=[10,8])
        vis.hist(data, alpha=0.8, bins=40) # creating a histogram
        #plt.grid(axis='y', alpha=0.6) # grid
            
    elif visual_type=='boxplot':
        fig,vis=plt.subplots(figsize=[8,4])
        vis.boxplot(data, vert=0) # creating a box plot
        #plt.grid(axis='x', alpha=0.6)
                     
    else:
        fig, vis=plt.subplots(figsize=[10,8])
        plt.scatter(x=data, y=data2, alpha=0.8, color='blue') # creating scatter

In [ ]:
# checking the distribution of goals in the seasons
goals_08_to_16=melted_match.groupby(['season', 'Team'])['goals'].sum() 
# plot histogram
plot_visual(data=goals_08_to_16, data2='', visual_type='hist')
plt.title('HISTOGRAM')
plt.xlabel('bins')
plt.ylabel('No of goals scored in the season');

The graph shows that the distribution of goals is right screwed. Further investigation can be shown on the boxplot

In [ ]:
# plot box plot
plot_visual(data=goals_08_to_16, data2='', visual_type='boxplot')
plt.title('SEASON 2008/2009 - 2015/2016 BOXPLOT')
plt.xlabel('No of goals scored');

The distribution is rght skewed but with severa outliers

### Research Question 1 What is the league table for the 4 major leagues in europe during 2015/2016 season? 

In [ ]:
melted_match['league_name'].value_counts()

In [1044]:
# Processing the data
columns_order = ['Pos', 'season', 'league_name', 'Club', 'MP', 'W', 'D', 'L', 'Pts', 'GF', 'GA', 'GD', 'avg_goals']
filtered_columns= ['Pos', 'Club', 'MP', 'W', 'D', 'L', 'Pts', 'GF', 'GA', 'GD', 'avg_goals']

def league_table(league):
    # Filter the data for the given league and season
    df_15_16 = (
        melted_match
        .query('season == "2015/2016" and league_name == @league')  
        .groupby(['season', 'league_name', 'Team'], as_index=False)
        .agg({
            'match_api_id': 'count',  # Total matches played
            'pts': [
                lambda x: (x == 3).sum(),  # Wins
                lambda x: (x == 1).sum(),  # Draws
                lambda x: (x == 0).sum(),  # Losses
                'sum'  # Total points
            ],
            'GF': 'sum',
            'GA': 'sum',
            'GD': 'sum',
            'goals': 'mean',  # Average goals
        })
    )

    # Flatten the column names
    df_15_16.columns = [
        'season', 'league_name', 'Club', 'MP', 
        'W', 'D', 'L', 'Pts', 'GF', 'GA', 'GD', 'avg_goals'
    ]

    # Sort the result by Pts and GD
    df_15_16 = df_15_16.sort_values(by=['Pts', 'GD'], ascending=[False, False])

    # Add a Rank column
    df_15_16['Pos'] = df_15_16.reset_index().index + 1
    # Reorder columns 
    df_15_16 = df_15_16[columns_order].reset_index()
    # filter columns
    df_15_16  = df_15_16[filtered_columns]


    return df_15_16

#### (a) England Premier League Table for 2015/2016 season

In [ ]:
league_table(league="England Premier League")

#### (b) France Ligue 1 Table for 2015/2016 season

In [ ]:
league_table(league="France Ligue 1")

#### (c) Spain LIGA BBVA Table for 2015/2016 season

In [ ]:
league_table(league="Spain LIGA BBVA")

#### (d) Italy Serie A Table for 2015/2016 season

In [ ]:
league_table(league="Italy Serie A")

### Saving for Google Looker Studio Analysis

In [ ]:
df_google_looker = (
    melted_match
    #.query('season == "2009/2010" and league_name == "England Premier League"')  # Filter for the specified season and league
    .groupby(['season', 'league_name', 'Team'], as_index=False)
    .agg({
        'match_api_id': 'count',  # Total matches played
        'pts': [
            lambda x: (x == 3).sum(),  # Wins
            lambda x: (x == 1).sum(),  # Draws
            lambda x: (x == 0).sum(),  # Losses
            'sum'  # Total points
        ],
        'GF': 'sum',
        'GA': 'sum',
        'GD': 'sum',
        'goals': 'mean',  # Average goals
    })
)

# Flatten the column names
df_google_looker.columns = [
    'season', 'league_name', 'Club', 'MP', 
    'W', 'D', 'L', 'Pts', 'GF', 'GA', 'GD', 'avg_goals'
]

# Sort the result by total_pts
df_google_looker = df_google_looker.sort_values(by=['Pts', 'GD'], ascending=[False, False])

df_google_looker.reset_index()

In [1050]:
# Saving
df_google_looker.to_csv('league_table_for_lookerStudio.csv', index_label='Id')

### Research Question 2 Which are the top 3 teams for each league in 2015/2016 season? 

In [ ]:
# Processing
top_3_teams = (
    melted_match
    .query('season=="2015/2016"')  
    .groupby(['season', 'league_name', 'Team'], as_index=False)['pts'] 
    .sum()
    .sort_values(by=['season', 'league_name', 'pts'], ascending=[True, True, False])
    .groupby(['season', 'league_name'], as_index=False)
    .head(3)
)

top_3_teams

In [ ]:
plt.figure(figsize=(9, 7))

# Create a bar plot
sns.barplot(data=top_3_teams, x='pts', y='Team', hue='league_name', dodge=False)

# Add labels and title
plt.title('Top 3 Teams in Each League (2015/2016 Season)', fontsize=16)
plt.xlabel('Points', fontsize=12)
plt.ylabel('Teams', fontsize=12)

# Add horizontal lines to separate leagues
league_boundaries = top_3_teams.groupby('league_name')['Team'].count().cumsum().values[:-1]
for boundary in league_boundaries:
    plt.axhline(y=boundary - 0.5, color='black', linestyle='--', linewidth=1)

# Add a legend
plt.legend(title='League', fontsize=10, title_fontsize=10, loc='upper left', bbox_to_anchor=(1, 1))

# Show the plot
plt.tight_layout() 
plt.show()

### Research Question 3 What teams improved the most over the time period? 

- We use Year-Over-Year Improvements (YoY)
- The teams played different number of matches. We normalize Points Based on Total Matches Played
- Then divide the points by the total matches played in each season. 

In [ ]:
# get the total points for each team per season
match_imp08_16=melted_match.query(
    'season in ["2008/2009", "2009/2010", "2010/2011", "2011/2012", "2012/2013", "2013/2014", "2014/2015", "2015/2016"]'
    ).groupby(
        ['season', 'Team'])['pts'].sum().unstack('season')

match_imp08_16

In [ ]:
# get the total matches played for each team per season
match_impp08_16=melted_match.query(
    'season in ["2008/2009", "2009/2010", "2010/2011", "2011/2012", "2012/2013", "2013/2014", "2014/2015", "2015/2016"]'
    ).groupby(
        ['season', 'Team'])['match_api_id'].count().unstack('season')

match_impp08_16

In [ ]:
# Divide points by number of matches played
df_normalized=match_imp08_16/match_impp08_16
df_normalized.dropna(how='any', inplace=True)
df_normalized

In [ ]:
# Calculate YoY improvements

'''# Difference between consecutive seasons
yoy_improvements = df_normalized.diff(axis=1)  

# Add a column: total YoY improvement
yoy_improvements['Total_YoY_Improvement'] = yoy_improvements.sum(axis=1)

# Team with the maximum YoY improvement
most_consistent_team = yoy_improvements['Total_YoY_Improvement'].idxmax()
max_yoy_improvement = yoy_improvements.loc[most_consistent_team, 'Total_YoY_Improvement']

# Result
print(f"The most consistent improver is {most_consistent_team} with a total YoY improvement of {max_yoy_improvement} points.");'''

In [ ]:
# Calculate YoY improvements
df_normalized['Total_YoY_Improvement']=(
    (df_normalized['2009/2010'] - df_normalized['2008/2009']) +
    (df_normalized['2010/2011'] - df_normalized['2009/2010']) +
    (df_normalized['2011/2012'] - df_normalized['2010/2011']) +
    (df_normalized['2012/2013'] - df_normalized['2011/2012']) +
    (df_normalized['2013/2014'] - df_normalized['2012/2013']) +
    (df_normalized['2014/2015'] - df_normalized['2013/2014']) +
    (df_normalized['2015/2016'] - df_normalized['2014/2015'])
    )

df_normalized[df_normalized['Total_YoY_Improvement']>0].reset_index().sort_values(by='Total_YoY_Improvement', ascending=False)

In [ ]:
# improve
df0=df_normalized[df_normalized['Total_YoY_Improvement']>0].reset_index().sort_values(by='Total_YoY_Improvement', ascending=False)
# drop
df00=df_normalized[df_normalized['Total_YoY_Improvement']<0].reset_index().sort_values(by='Total_YoY_Improvement', ascending=False)

plt.figure(figsize=(12, 7))

# chart 1 improve
plt.subplot(1, 2, 1)  # 1 row, 2 columns, position 1
sns.barplot(data=df0, y='Team', x='Total_YoY_Improvement',dodge=False, palette='viridis')
plt.title('Most Improved Teams Accross Seasons', fontsize=16)
plt.xlabel('Total_YoY_Improvement', fontsize=12)
plt.ylabel('Teams', fontsize=12)

# chart 2 drop
plt.subplot(1, 2, 2)  # 1 row, 2 columns, position 2
sns.barplot(data=df00, y='Team', x='Total_YoY_Improvement',dodge=False, palette='viridis_r')
plt.title('Teams with Most Decline Across Seasons', fontsize=16)
plt.xlabel('Total_YoY_Improvement', fontsize=12)
plt.ylabel('') # remove duplicate axes plt.ylabel('Teams', fontsize=12)

# Show the plot
plt.tight_layout()
plt.show();

##### The most consistent improver is Napoli with a total YoY improvement of 0.9473684210526316 points.

### Insight 1: Team Playing Style Analysis
- Identify unique playing styles for teams

In [ ]:
# Processing

# Team attributes against goals and points obtained
class_list =[]
for i in melted_match.columns:
    if 'Class' not in i:
        class_list.append(i)
class_list

In [1060]:
exclude_list=['country_name', 'stage', 'date', 'match_api_id', 'home_team_goal', 'away_team_goal', 'results', 'Location']
req_columns=[
 'season',
 'league_name',
 'Team',
 'buildUpPlaySpeed',
 'buildUpPlayDribbling',
 'buildUpPlayPassing',
 'chanceCreationPassing',
 'chanceCreationCrossing',
 'chanceCreationShooting',
 'defencePressure',
 'defenceAggression',
 'defenceTeamWidth',
 'goals',
 'pts',
 'GF',
 'GA',
 'GD']
num_columns=[ 'buildUpPlaySpeed',
 'buildUpPlayDribbling',
 'buildUpPlayPassing',
 'chanceCreationPassing',
 'chanceCreationCrossing',
 'chanceCreationShooting',
 'defencePressure',
 'defenceAggression',
 'defenceTeamWidth']

In [ ]:
melted_match[req_columns]

In [ ]:
melted_class = (
    melted_match.groupby(['season', 'league_name', 'Team'] + num_columns, as_index=False) 
    .agg({
        'match_api_id': 'count',  # Total matches played
        'pts': [
            lambda x: (x == 3).sum(),  # Wins
            lambda x: (x == 1).sum(),  # Draws
            lambda x: (x == 0).sum(),  # Losses
            'sum'  # Total points
        ],
        'GF': 'sum',
        'GA': 'sum',
        'GD': 'sum',
        'goals': 'mean',  # Average goals
    })
)

# Flatten the column names
melted_class.columns = [
    'season', 'league_name', 'Team'] + num_columns + ['MP', 
    'W', 'D', 'L', 'Pts', 'GF', 'GA', 'GD', 'Avg_Goals']

# Sort the result by total_pts
melted_class = melted_class.sort_values(by=['season', 'league_name', 'Pts'], ascending=[True, True, False])

melted_class

#### Normalize the numeric columns to ensure equal weighting during clustering.
- Use the relevant columns for clustering

In [1063]:
# Select numeric columns for analysis
numeric_columns = [
    'buildUpPlaySpeed', 'buildUpPlayDribbling', 'buildUpPlayPassing', 
    'chanceCreationPassing', 'chanceCreationCrossing', 'chanceCreationShooting',
    'defencePressure', 'defenceAggression', 'defenceTeamWidth'
]

# Normalize the data
scaler = MinMaxScaler()
melted_class_normalized = melted_class.copy()
melted_class_normalized[numeric_columns] = scaler.fit_transform(melted_class[numeric_columns])

In [ ]:
# Define the number of clusters
num_clusters = 4

# Perform K-means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
melted_class_normalized['Cluster'] = kmeans.fit_predict(melted_class_normalized[numeric_columns])

# Add cluster labels to the original data
melted_class['Cluster'] = melted_class_normalized['Cluster']

# Display the final dataframe
melted_class


#### Analyzing Clusters
- Calculate the average values of numeric columns for each cluster to interpret playing styles.

In [ ]:
cluster_analysis = (
    melted_class
    .groupby('Cluster')[numeric_columns]
    .mean()
    .reset_index()
)

cluster_analysis

In [ ]:
# Define the cluster labels
cluster_labels = {
    0: 'Balanced',
    1: 'Balanced-to-Offensive',
    2: 'Defensive',
    3: 'Offensive'  
}

# Rename clusters
cluster_analysis['Cluster'] = cluster_analysis['Cluster'].map(cluster_labels)
cluster_analysis

#### Radar Chart Visualization
- Visualize the playing styles for each cluster using radar charts
- Offensive Playstyle: High buildUpPlayPassing, chanceCreationPassing, chanceCreationShooting. Low defencePressure and defenceAggression.
- Defensive Playstyle: High defencePressure, defenceAggression, and defenceTeamWidth. Low offensive attributes.
- Balanced Playstyle: Moderate values across both offensive and defensive attributes.

In [ ]:
# Attributes
attributes = [
    'buildUpPlaySpeed', 'buildUpPlayDribbling', 'buildUpPlayPassing',
    'chanceCreationPassing', 'chanceCreationCrossing', 'chanceCreationShooting',
    'defencePressure', 'defenceAggression', 'defenceTeamWidth'
]

# Angles for radar chart
angles = np.linspace(0, 2 * np.pi, len(attributes), endpoint=False).tolist()
angles += angles[:1]  # Close the circle

# Iterate through each cluster
for cluster_label in cluster_analysis['Cluster'].unique():
    # Filter data for the current cluster
    cluster_data = cluster_analysis[cluster_analysis['Cluster'] == cluster_label]
    
    # Extract values
    cluster_values = cluster_data[attributes].values[0].tolist()
    cluster_values += cluster_values[:1]

    # Plot 
    plt.figure(figsize=(6, 6))
    plt.polar(angles, cluster_values, marker='o', color='blue', label=cluster_label)
    plt.fill(angles, cluster_values, color='blue', alpha=0.25)
    
    # Add labels
    plt.thetagrids(np.degrees(angles[:-1]), labels=attributes, fontsize=10)
    plt.title(f'{cluster_label} Cluster Playing Style', size=14, color='blue', pad=20)
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.2))

    plt.show();


####  Comparing Clusters with Performance Metrics

In [ ]:
# Merge clusters with performance metrics
performance_analysis = melted_class[['Team', 'Cluster', 'Pts', 'GF', 'GA', 'GD']]

# Group by cluster and analyze average performance
performance_by_cluster = (
    performance_analysis
    .groupby('Cluster')[['Pts', 'GF', 'GA', 'GD']]
    .mean()
    .reset_index()
)

performance_by_cluster

In [1069]:
# Define the cluster labels
cluster_labels = {
    0: 'Offensive',
    1: 'Balanced',
    2: 'Defensive-to-Balanced',
    3: 'Weak Defensive'  
}

# Rename the clusters
performance_by_cluster['Cluster'] = performance_by_cluster['Cluster'].map(cluster_labels)

#### Visualize performance differences using bar chart.

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(data=performance_by_cluster.melt(id_vars='Cluster'), x='Cluster', y='value', hue='variable')

plt.title('Performance Metrics by Cluster', fontsize=16)
plt.xlabel('Cluster')
plt.ylabel('Average Value')
plt.legend(title='Metric')

plt.tight_layout()
plt.show();

#### Conclusion

- Radar charts helps to identify the offensive, defensive, or balanced styles of each cluster.
- From the performance metrics, offensive and balanced styles are the most effective playing styles

### Insight 2: Analysisng the correlation of Attributes with Performance
- Determine how specific attributes impact performance

In [ ]:
# Select numeric columns for analysis
numeric_columns = ['buildUpPlaySpeed', 'buildUpPlayDribbling', 'buildUpPlayPassing', 
                   'chanceCreationPassing', 'chanceCreationCrossing', 'chanceCreationShooting', 
                   'defencePressure', 'defenceAggression', 'defenceTeamWidth', 
                   'GF', 'GA', 'GD', 'pts', 'goals']

# Filter relevant columns
correlation_data = melted_match[numeric_columns]

# Correlation matrix
correlation_matrix = correlation_data.corr()
correlation_matrix # table

#### Visualize the correlation matrix in a heatmap

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='cividis_r', cbar=True)
plt.title('Correlation Between Class Attributes and Performance Metrics')

plt.show();

#### Visualize the correlation Between Build-Up Play Passing and Goals Scored on a Scatter

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=melted_class, x='buildUpPlayPassing', y='GF')

plt.title('Correlation Between Build-Up Play Passing and Goals Scored (GF)')
plt.xlabel('Build-Up Play Passing')
plt.ylabel('Goals Scored (GF)')
plt.show()

#### Regression for buildUpPlayPassing and GF

In [ ]:
X = melted_class[['buildUpPlayPassing']]
y = melted_class['GF']

# Add a constant for the regression
X = sm.add_constant(X)

# Fit the model
model = sm.OLS(y, X).fit()

# Display results
model.summary()

### Insight 3: League-Level Style Analysis
- Assess the dominant strategies in different leagues.

In [1075]:
# Style-related columns
style_columns = [
    'buildUpPlaySpeed', 'buildUpPlayDribbling', 'buildUpPlayPassing', 
    'chanceCreationPassing', 'chanceCreationCrossing', 'chanceCreationShooting', 
    'defencePressure', 'defenceAggression', 'defenceTeamWidth'
]
league_style_analysis = melted_match.groupby(['season','league_name'])[style_columns].mean().reset_index()

league_style_analysis.to_csv('league_style_analysis_for_lookerStudio.csv', index_label='Id')


#### Bar Plot

In [ ]:
# Melt the data
league_style_melted = league_style_analysis.melt(
    id_vars=['season', 'league_name'], 
    var_name='Style Attribute', 
    value_name='Average Value'
)

# Bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=league_style_melted, y='Average Value', x='Style Attribute', hue='league_name', palette='Paired')

plt.title('Average Playing Styles Across Leagues', fontsize=16)
plt.xlabel('Style Attribute', fontsize=12)
plt.ylabel( 'Average Value', fontsize=12)
plt.xticks(rotation=90)

plt.legend(title='League', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10) 
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show();

#### Heatmap

In [ ]:
# Plot heatmap
plt.figure(figsize=(12, 6))

league_style_analysis = melted_match.groupby('league_name')[style_columns].mean().reset_index()
sns.heatmap(league_style_analysis.set_index('league_name'), annot=True, fmt='.2f', cmap='viridis')

plt.title('Heatmap of Average Playing Styles by League', fontsize=16)
plt.xlabel('Style Attribute', fontsize=12)
plt.ylabel('League', fontsize=12)
plt.tight_layout()
plt.show()

#### Conclusion

- Italy Serie A, Scotland Premeir League and England Premier League have higher buildUpPlaySpeed, indicating a faster playing style.
- Italy Serie A focuses more on chanceCreationPassing and chanceCreationShooting.


#### Insight 4: Summary Statistics for players

##### (a) Summary statistics for numeric columns

In [1085]:
player_info

In [ ]:
numeric_columns = ['height', 'weight', 'overall_rating', 'potential']
summary_stats = player_info[numeric_columns].describe().T


In [ ]:
summary_stats['median'] = player_info[numeric_columns].median()
summary_stats

##### (b) Distribution of the preferred_foot

In [ ]:
foot_distribution = player_info['preferred_foot'].value_counts()

# Plot a bar chart
sns.barplot(x=foot_distribution.index, y=foot_distribution.values, palette='pastel')
plt.title('Distribution of Preferred Foot', fontsize=16)
plt.xlabel('Preferred Foot', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.show()

##### (c) Attacking and Defensive work rates

In [ ]:
attacking_work_rate_dist = player_info['attacking_work_rate'].value_counts()
defensive_work_rate_dist = player_info['defensive_work_rate'].value_counts()

print("Attacking Work Rate Distribution:")
print(attacking_work_rate_dist)

print("\nDefensive Work Rate Distribution:")
print(defensive_work_rate_dist)


In [ ]:
# Combine the two distributions for visualization
work_rate_data = pd.DataFrame({
    'Attacking Work Rate': attacking_work_rate_dist,
    'Defensive Work Rate': defensive_work_rate_dist
}).T

# Create a grouped bar chart
work_rate_data.plot(kind='bar', figsize=(10, 6), color=['skyblue', 'lightcoral'])

plt.title('Distribution of Work Rates', fontsize=16)
plt.xlabel('Work Rate Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.legend(title='Work Rate Level', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


#### Insight 5: Tracking trends of the Player Attributes over time

In [ ]:
# Change 'date' column is in datetime format
#player_info['date'] = pd.to_datetime(player_info['date'])
player_info

In [930]:
# Players attribute categories

Overall_Performance= ['overall_rating', 'potential']

Physical_Attributes= ['acceleration', 'sprint_speed', 'agility', 'balance', 'stamina', 'strength', 'jumping']

Technical_Skills= ['dribbling', 'ball_control', 'short_passing', 'long_passing', 'crossing', 'finishing', 'heading_accuracy',
                   'volleys', 'curve', 'free_kick_accuracy', 'long_shots', 'shot_power']

Defensive_Skills= ['interceptions', 'marking', 'standing_tackle', 'sliding_tackle']

Mental_Attributes= ['reactions', 'vision', 'positioning', 'aggression']

Goalkeeping_Attributes= ['gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']

Others= ['preferred_foot', 'attacking_work_rate', 'defensive_work_rate', 'age']

In [931]:
# Calculate the attributes and group by 'date' 
attributes = ['overall_rating', 'dribbling', 'crossing', 'long_passing', 'finishing', 'free_kick_accuracy', 'agility', 'ball_control']
trend_data = player_info.groupby('date')[attributes].mean().reset_index()


In [ ]:
# Plot the trends
plt.figure(figsize=(12, 8))
for attribute in attributes:
    sns.lineplot(data=trend_data, x='date', y=attribute, label=attribute)

plt.title('Trend Analysis of Player Attributes Over Time', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Average Value', fontsize=12)
plt.legend(title='Attributes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# If the data points are too granular, you can resample the data to a monthly or yearly frequency to smooth out the trends.
# Resample to monthly averages (if there are multiple records per month)
trend_data_monthly = trend_data.set_index('date').resample('M')[attributes].mean().reset_index()

# Plotting the resampled data
plt.figure(figsize=(12, 8))
for attribute in attributes:
    sns.lineplot(data=trend_data_monthly, x='date', y=attribute, label=attribute)

plt.title('Monthly Trend Analysis of Player Attributes', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Average Value', fontsize=12)
plt.legend(title='Attributes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
for i in player_info['player_name'].unique():
    if 'Ronaldo' in i:
        print(i)

In [ ]:
# Analyzing Player Development
# To track individual player development, you can follow a similar approach but group by player_name or player_api_id 
# and aggregate data for each player over time
# Group by 'date' and 'player_name' to track individual player trends
player_trends = player_info.groupby(['date', 'player_name'])[attributes].mean().reset_index()

# Plot for one specific player (e.g., a player named "Lionel Messi")
player_trends[player_trends['player_name'] == 'Lionel Messi'].plot(x='date', y=attributes, figsize=(12, 8), marker='o')
plt.title('Player Development for Lionel Messi', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Attribute Value', fontsize=12)
plt.legend(title='Attributes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
# Analyzing Player Development
# To track individual player development, you can follow a similar approach but group by player_name or player_api_id 
# and aggregate data for each player over time
# Group by 'date' and 'player_name' to track individual player trends
player_trends = player_info.groupby(['date', 'player_name'])[attributes].mean().reset_index()

# Plot for one specific player (e.g., a player named "Lionel Messi")
player_trends[player_trends['player_name'] == 'Ronaldo'].plot(x='date', y=attributes, figsize=(12, 8), marker='o')
plt.title('Player Development for Cristiano Ronaldo', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Attribute Value', fontsize=12)
plt.legend(title='Attributes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


Player Age and Potential
Use birthday to calculate player age and analyze:
Age distribution.
Relationship between age and potential or overall_rating.
Peak performance age for players.

In [937]:
# Ensure 'birthday' column is in datetime format
player_info['birthday'] = pd.to_datetime(player_info['birthday'])

# Calculate player's age
player_info['age'] = (pd.to_datetime('2016-05-30') - player_info['birthday']).astype('<m8[Y]')


In [ ]:
# Plot age distribution (Histogram)
plt.figure(figsize=(10, 6))
sns.histplot(player_info['age'], kde=True, bins=20, color='skyblue')
plt.title('Age Distribution of Players', fontsize=16)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()


Relationship Between Age and Potential / Overall Rating
You can use scatter plots to examine how age relates to potential and overall_rating.

In [ ]:
#Age vs. Potential
plt.figure(figsize=(10, 6))
sns.scatterplot(data=player_info, x='age', y='potential', color='orange')
plt.title('Age vs Potential', fontsize=16)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Potential', fontsize=12)
plt.show()


In [ ]:
#Age vs. Overall Rating
plt.figure(figsize=(10, 6))
sns.scatterplot(data=player_info, x='age', y='overall_rating', color='green')
plt.title('Age vs Overall Rating', fontsize=16)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Overall Rating', fontsize=12)
plt.show()


Peak Performance Age for Players
To find the peak performance age, you can aggregate the data by age and calculate the average of potential and overall_rating. Then, you can identify the age with the highest average ratings.

In [ ]:
# Group by age and calculate the mean of potential and overall_rating
age_performance = player_info.groupby('age')[['potential', 'overall_rating']].mean().reset_index()

# Plot Age vs Average Ratings
plt.figure(figsize=(12, 8))
sns.lineplot(data=age_performance, x='age', y='potential', label='Potential', color='orange', linewidth=2)
sns.lineplot(data=age_performance, x='age', y='overall_rating', label='Overall Rating', color='green', linewidth=2)

# Highlight peak performance ages
peak_performance_age = age_performance.loc[age_performance['overall_rating'].idxmax(), 'age']
plt.axvline(x=peak_performance_age, color='red', linestyle='--', label=f'Peak Performance Age: {peak_performance_age}')

plt.title('Age vs Average Potential and Overall Rating', fontsize=16)
plt.xlabel('Age', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.legend(title='Attributes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


perform Attribute Clustering using K-Means:

In [942]:



# Select relevant features (example)
attributes = ['sprint_speed', 'crossing', 'vision', 'short_passing', 'dribbling', 'ball_control']

# Subset the data
player_attributes = player_info[attributes]

# Standardize the data (important for clustering)
scaler = StandardScaler()
scaled_attributes = scaler.fit_transform(player_attributes)



In [ ]:
# Perform K-Means clustering
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
player_info['cluster'] = kmeans.fit_predict(scaled_attributes)

# Check the cluster centers (means of the attributes for each cluster)
cluster_centers = pd.DataFrame(scaler.inverse_transform(kmeans.cluster_centers_), columns=attributes)
print(cluster_centers)


In [ ]:
# Define the archetypes based on the cluster centers
archetypes = {
    0: 'Speedy Winger',
    1: 'Playmaker',
    2: 'All-rounder'
}

# Map the cluster labels to archetypes
player_info['archetype'] = player_info['cluster'].map(archetypes)

# Show players with their archetypes
print(player_info[['player_name', 'archetype']].head())


In [ ]:
'''import matplotlib.pyplot as plt
import seaborn as sns

# For 2D visualization, we can use the first two principal components (PCA) of the scaled data
from sklearn.decomposition import PCA'''

pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_attributes)

# Create a DataFrame for plotting
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
pca_df['archetype'] = player_info['archetype']

# Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=pca_df, x='PC1', y='PC2', hue='archetype', palette='Set1', s=100)
plt.title('Player Archetypes Based on Skills', fontsize=16)
plt.xlabel('Principal Component 1', fontsize=12)
plt.ylabel('Principal Component 2', fontsize=12)
plt.legend(title='Archetype', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


## Conclusions
The soccer database has five datasets, league, country, player, player attribute, team and team attribute. It is a detailed dabase for European major leagues covering several seasons from 2008/2009 t0 2015/2016. 

The project seeks to answer three questions, what teams improved the most over the time period, which players had the most penalties and which was the the most preferred leg for penalty-takers in 2016 among the players who scored more than the mean penalties in that year?

In attempting to find solutions to the question, each dataset was examineed for inconsistencies, colomn names, corrected, missing values replace or droped in certain datasets before they were finally merged and cleaned. Visual presentations created and inteprated.

From the analysis and visualization, Richie Lambert is the player who scored most of the penalties. I also found that Paris Saint-Germain is the most improved team over the period of time given, followed by Napoli and Cracovia being the in the third position. Moreover, the findings also indicate that most of the penalty takers in 2016 preferred right leg compared to the right leg. The findings also shows that the distribution of the number of goals scored in the two seasons are right skewed.

Whereas I was able to show that there is a correltaion between the number of goals scored in the two extreme seasons (2008/2009 and 2015/2016), theer are  other seasons that were not considered. There is likelihood that a team that improved between the two seasons might not have improved in the seasons prior 2015/2016. Goal difference between the two seasons was used as a measured of improvement in performance because the ultimate objective of team managers, players and teams is to improve to score goals, but there could be criteria for measuring performance.

In [946]:
#from subprocess import call
#call(['python', '-m', 'nbconvert', 'Investigate_a_Dataset.ipynb'])